In [1]:
cd(@__DIR__)
using Pkg
Pkg.activate(".")

# using Plots
using Optim
using LinearAlgebra
using Random
using Distributions
using GLMakie

  Activating new project at `c:\Documents\Project\metadynamics`

In [2]:
function meshgrid(x, y)
    ny, nx = length(y), length(x)
    rx = repeat(reshape(x, 1, nx), ny, 1)
    ry = repeat(reshape(y, ny, 1), 1, nx)
    return rx, ry
end

function elementwise_f(f, X, Y)
    # Check if X and Y have the same dimensions
    if size(X) != size(Y)
        error("Input arrays X and Y must have the same dimensions.")
    end

    # Initialize the output array Z with zeros
    Z = zeros(size(X))

    # Apply f(x, y) element-wise and store in Z
    for i = 1:size(X, 1)
        for j = 1:size(X, 2)
            x = X[i, j]
            y = Y[i, j]
            z = f([x, y])
            Z[i, j] = z
        end
    end

    # Return the result
    return Z
end

elementwise_f (generic function with 1 method)

In [3]:
n_grid = 480
x = range(-1.2, stop=1.2, length=n_grid+1)
y = range(-1.2, stop=1.2, length=n_grid+1)

# Create 2D meshgrid
X, Y = meshgrid(x, y)

# Compute Z as a function of X and Y
# Define a common frequency for the landscape
frequency = 2 * π * 1

# A simple PES function
function f_basin(x)
    coord = [x..., 0.0]
    rho_basin = 0.24 * sum((coord - [0.0, 0.0, 0.0]).^4)
    return rho_basin
end

function f_z(x)
    return sin.(frequency * x[1]) .* cos.(frequency * x[2]) + f_basin(x)
end

# Compute Z as a function of X and Y using the common frequency
Z = elementwise_f(f_z, X, Y) + elementwise_f(f_basin, X, Y)

# Penalty function
function f_phi_p(x, x_0, sigma=0.01, W=0.5)
    phi_p = sum([W * exp(-sum((x - c).^2) / (2*sigma^2)) for c in x_0])
    return phi_p
end

f_phi_p (generic function with 3 methods)

In [4]:
x_0 = [-0.25, 0.01]
# x_0 = [-0.1, 0.1]
lower_bounds = [-1.0, -1.0]
upper_bounds = [1.0, 1.0]

inner_optimizer = LBFGS()
opti = optimize(x -> f_z(x), lower_bounds, upper_bounds, x_0, Fminbox(inner_optimizer), Optim.Options(g_tol = 1e-6))
x_i = Optim.minimizer(opti)

2-element Vector{Float64}:
 -0.2496217672537466
 -1.604668770529158e-10

In [5]:
# Initialize
basin_list = [x_i]
x_list = [x_i]
rho_new = copy(Z)  # Assuming Z is the initial state similar to rho_bg in Python
sigma = 0.03
W = 0.2

n_trial = 500
for i in 1:n_trial
    # Activation step
    n_p = min(i, n_trial)  # Ensure we don't go beyond the length of x_list
    start_index = max(1, length(x_list) - n_p + 1)
    f_new = x -> f_z(x) + f_basin(x) + f_phi_p(x, x_list, sigma, W) 
    
    # Relaxation step
    d_b = 0.5
    x_min = max(x_list[end][1] - d_b, -10)  
    y_min = max(x_list[end][2] - d_b, -10)
    x_max = min(x_list[end][1] + d_b, 10)
    y_max = min(x_list[end][2] + d_b, 10)

    lower_bounds = [x_min, y_min]
    upper_bounds = [x_max, y_max]

    x_init = x_list[end] + (rand(2).-0.5)*0.001

    inner_optimizer = BFGS()
    opti_phi_p = optimize(f_new, lower_bounds, upper_bounds, x_init, 
                          Fminbox(inner_optimizer), Optim.Options(g_tol = 1e-9))
    x_new = Optim.minimizer(opti_phi_p)

    # if x_list[end] != x_new
    #     println(x_list)
    # end

    # Confirm the sampling of a new local minimum
    if f_phi_p(x_new, x_list, sigma, W) < W * 0.0001
        push!(basin_list, x_new)
    end

    push!(x_list, x_new)
end

In [6]:
basin_list

8-element Vector{Vector{Float64}}:
 [-0.2496217672537466, -1.604668770529158e-10]
 [0.24924642704954153, -0.4941309087971097]
 [0.7309603349075954, -1.029950446239997e-12]
 [0.24970923399604014, 0.49413096288575176]
 [-0.7309480992607653, 0.49409011660528973]
 [-0.24921866992685807, 0.9568651041056584]
 [1.1688004323145307, -0.4933075922428859]
 [0.692257701478393, -0.9541775487447799]

In [7]:
# Assuming X, Y, Z are defined as before for the surface plot

# Extract the X and Y coordinates from the basin filling process
basin_list_x = [point[1] for point in basin_list]
basin_list_y = [point[2] for point in basin_list]

# Calculate the corresponding Z values for each point in x_list and y_list
basin_list_z = [f_z(x) + f_basin(x) for x in zip(basin_list_x, basin_list_y)]

# Extract the X and Y coordinates from the explored coordinates
history_x = [point[1] for point in x_list]
history_y = [point[2] for point in x_list]

# Calculate the corresponding Z values for each point in x_list and y_list
history_z = [f_z(x) + f_basin(x) for x in zip(history_x, history_y)]


501-element Vector{Float64}:
 -0.9981334973674625
 -0.9368595394497691
 -0.9358146142768097
 -0.9224961097634904
 -0.8532745220901997
 -0.9285048149815005
 -0.8518865686617956
 -0.7696790069005395
 -0.7052701944242626
 -0.8744808403725808
  ⋮
 -0.16208364879162845
 -0.023427591588774725
 -0.1514983939480167
 -0.843969271991355
  0.007041661903276935
 -0.9120532292512786
 -0.8394536108950036
 -0.38830473007687893
 -0.3996536003445689

In [8]:
# Plot the PES and explored minima
f = Figure()
ax = Axis3(f[1, 1])

surface!(ax, X, Y, Z, colormap = :viridis, shading = NoShading)
scatter!(ax, basin_list_x, basin_list_y, basin_list_z.+0.01, color = :red, markersize = 12)
lines!(ax, history_x, history_y, history_z.*0, color = :red, markersize = 0, linewidth = 1)

f

In [ ]:
# Plot the PES with bias potential
f2 = Figure()
ax = Axis3(f2[1, 1])

f_new = x -> f_z(x) + f_basin(x) + f_phi_p(x, x_list, sigma, W) 
Z_bias = elementwise_f(f_new, X, Y)

surface!(ax, X, Y, Z_bias, colormap = :viridis, shading = NoShading)

f2